In [ ]:
import array, calendar, csv, json, math, random, time

def LonLatToPixelXY(lonlat):
    (lon, lat) = lonlat
    x = (lon + 180.0) * 256.0 / 360.0
    y = 128.0 - math.log(math.tan((lat + 90.0) * math.pi / 360.0)) * 128.0 / math.pi
    return [x, y]
# p0 = LonLatToPixelXY(districts_of_origin[origin]['coordinates'])


def FormatEpoch(datestr, formatstr):
    return calendar.timegm(time.strptime(datestr, formatstr))
#FormatEpoch("%s-%s-%s" % (year0, month0, day0), '%Y-%m-%d')
def GetPoint(row, epsilon):
    p0 = LonLatToPixelXY(row['org'])
    p2 = LonLatToPixelXY(row['dst'])
    p1 = [0,0]
    p1[0] = ((p2[0] + p0[0]) / 2)
    p1[1] = ((p2[1] + p0[1]) / 2)
    p1[0] += random.choice([1,-1]) * random.random() * 4
    p1[1] += random.choice([1,-1]) * random.random() * 4
    e0 = FormatEpoch("%s-%s-%s" % (int(row["year"]) - 1, random.randint(2,11), random.randint(1,28)), '%Y-%m-%d')
    e1 = FormatEpoch("%s-%s-%s" % (int(row["year"]), random.randint(1,12), random.randint(1,28)), '%Y-%m-%d')
    return [p0[0], p0[1], p1[0], p1[1], p2[0], p2[1], e0, e1]

In [ ]:
raw = []
with open("../data/making-a-point-flow-map/comtrade.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        raw.append(row)

In [ ]:
raw[0]

In [ ]:
with open("../data/making-a-point-flow-map/country-centroids.geojson") as f:
    data = json.load(f)

In [ ]:
data["features"][0]

In [ ]:
country_idx = {}
for feature in data["features"]:
    idx = feature["properties"]["iso_alpha-3"]
    country_idx[idx] = feature["geometry"]["coordinates"]


In [ ]:
time_series = []
for row in raw:
    if row['Reporter ISO'] in country_idx and row['Partner ISO'] in country_idx and row['Trade Flow'] == 'Export':
        time_series.append({'org': country_idx[row['Reporter ISO']],
                            'dst': country_idx[row['Partner ISO']],
                            'year': row['Year'],
                            'value': float(row['Trade Value (US$)'])
                           })

In [ ]:
totals = {'2014': 0, '2015': 0, '2016': 0, '2017': 0}
for row in time_series:
        totals[row['year']] += row['value']

In [ ]:
totals

In [ ]:
# Each point will be 8 32bit floats so 256 bytes per animated points
# So lets say each point will be worth 20 million dollars


In [ ]:
scaling_factor = 10000000
points = []
for row in time_series:
    for i in range(0, int(row['value']/scaling_factor)):
        points += GetPoint(row, int(row['value']/scaling_factor))
array.array('f', points).tofile(open('../data/making-a-point-flow-map/us-exports.bin', 'wb'))    
len(points)/8

In [ ]:
!rsync -rcav ../data/making-a-point-flow-map timemachine2:/usr2/aberkley.earthtime.org/app/misc/

In [ ]:
print 'TRUE\tmaking_a_point_flow_map\tMaking EarthTime Maps\tPoint Flow\tCREATE Lab\t2013\t2018\t1\thttps://aberkley.earthtime.org/misc/making-a-point-flow-map/us-exports.bin\t\t\tpoint-flow\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t8\tWebGLVectorTile2.pointFlowVertexShader\tWebGLVectorTile2.pointFlowFragmentShader\tWebGLVectorTile2.prototype._drawPointFlow\t0.25\t0.5'